# k Nearest Neighbor Classifier for Hate Speech Detection

### Import Libraries

In [26]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from evaluate_classification import EvaluateBinaryClassification

### Initialise Random variables

In [4]:
SEED = 123
np.random.seed(SEED)

### Loading Data

In [5]:
BASE = 'D:\\ResearchDataGtx1060\\SentimentData\\Hate\\'
fins_train = ['random_hate_train.csv']
fins_test = ['eastasian_hate_test.csv']
track = 0

In [6]:
# We apply only this preprocessing because our data is already preprocessed
def cleanNonAscii(text):
    '''
    Remove Non ASCII characters from the dataset.
    Arguments:
        text: str
    returns: 
        text: str
    '''
    return ''.join(i for i in text if ord(i) < 128)

In [9]:
df_train = pd.read_csv(BASE+fins_train[track])
df_train.head()

,label,text
0,1,<user> if you are one of the <number> mil <has...
1,0,best <hashtag> law of attraction </hashtag> <h...
2,1,<hashtag> michelle obama </hashtag> is the mos...
3,0,smiling because life is good rite now ! <repea...
4,0,ã ¢ â  â ¤ ã ¯ â ¸ â  ã ¢ â  â ¤ ã ¯ â ¸ â ...


In [10]:
df_train.groupby('label').count()

,text
label,
0,2242
1,2242


In [11]:
df_train['text'] = df_train['text'].apply(cleanNonAscii)
df_train.head(5)

,label,text
0,1,<user> if you are one of the <number> mil <has...
1,0,best <hashtag> law of attraction </hashtag> <h...
2,1,<hashtag> michelle obama </hashtag> is the mos...
3,0,smiling because life is good rite now ! <repea...
4,0,<hashtag> ...


In [12]:
X_train, y_train = df_train['text'].values, df_train['label'].values

In [13]:
df_test = pd.read_csv(BASE+fins_test[track])
df_test.head()

,label,text
0,1,<user> <user> the chinese are probably sprayin...
1,0,rt <user> : unpatriotic losers are tweeting ou...
2,1,<user> thus <hashtag> 2019 n co v </hashtag> i...
3,0,north korea closes borders to avoid coronaviru...
4,1,<user> this is a declaration of war . it prove...


In [14]:
df_test.groupby('label').count()

,text
label,
0,3898
1,3898


In [15]:
df_test['text'] = df_test['text'].apply(cleanNonAscii)
df_test.head(5)

,label,text
0,1,<user> <user> the chinese are probably sprayin...
1,0,rt <user> : unpatriotic losers are tweeting ou...
2,1,<user> thus <hashtag> 2019 n co v </hashtag> i...
3,0,north korea closes borders to avoid coronaviru...
4,1,<user> this is a declaration of war . it prove...


In [16]:
X_test, y_test = df_test['text'].values, df_test['label'].values

### Transforming data suitable for model format

In [17]:
count_vectorizer = CountVectorizer(binary=False)
count_vectorizer.fit(X_train)
#count_vectorizer.vocabulary_

CountVectorizer()

In [18]:
train_vectors = count_vectorizer.transform(X_train)
X_train_vectors = train_vectors.toarray()
test_vectors = count_vectorizer.transform(X_test)
X_test_vectors = test_vectors.toarray()

### define and fit the model

In [19]:
k_range = list(range(1, 19))
param_grid = dict(n_neighbors=k_range)
scores = ['accuracy']

In [21]:
clf_KNeighborsClassifier = KNeighborsClassifier()
keys = clf_KNeighborsClassifier.get_params().keys()
print(keys)
grid_nbc = GridSearchCV(clf_KNeighborsClassifier, param_grid, scoring='%s' % scores[0], cv=10)
grid_nbc.fit(X_train_vectors, y_train)

dict_keys(['algorithm', 'leaf_size', 'metric', 'metric_params', 'n_jobs', 'n_neighbors', 'p', 'weights'])


GridSearchCV(cv=10, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                         13, 14, 15, 16, 17, 18]},
             scoring='accuracy')

In [22]:
print("Best parameters set found on development set:")
print(grid_nbc.best_params_)

Best parameters set found on development set:
{'n_neighbors': 1}



### Evaluating the model with test dataset

In [24]:
actual, predicted = y_test, grid_nbc.predict(X_test_vectors)

In [27]:
ebc = EvaluateBinaryClassification(gnd_truths = actual, predictions = predicted)
print(ebc.get_full_report())

EvaluateBinaryClassification Object Created

Total Samples	7796
Positive Samples	3898
Negative Samples	3898
True Positive	1596
True Negative	2525
False Positive	1373
False Negative	2302
Accuracy	0.5286044125192406
Precision	0.5375547322330751
Recall	0.409440738840431
F1 Measure	0.4648318042813455
Cohen Kappa Score	0.05720882503848124
Area Under Curve	0.5286044125192406

              precision    recall  f1-score   support

           0       0.52      0.65      0.58      3898
           1       0.54      0.41      0.46      3898

    accuracy                           0.53      7796
   macro avg       0.53      0.53      0.52      7796
weighted avg       0.53      0.53      0.52      7796



In [29]:
ebc.save_full_report(model_name='kNN', path='C:\\Users\\User\\JupyterPythonPredator\\COVID19\\domain_adaptation_rerun_randomhate_eastasianhate_')